<a href="https://colab.research.google.com/github/Mithunjha/ITK_ImageRegistration3D/blob/main/Registration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
import sys
!pip install itk
import itk 
from distutils.version import StrictVersion as VS

     |████████████████████████████████| 54.5 MB 1.4 MB/s 
     |████████████████████████████████| 15.0 MB 50.6 MB/s 
     |████████████████████████████████| 16.6 MB 300 kB/s 
     |████████████████████████████████| 20.3 MB 532 kB/s 
     |████████████████████████████████| 70.6 MB 7.5 kB/s 
     |████████████████████████████████| 95.3 MB 37 kB/s 


In [ ]:
if VS(itk.Version.GetITKVersion()) < VS("4.9.0"):
    print("ITK 4.9.0 is required.")
    sys.exit(1)

In [ ]:
# if len(sys.argv) != 3:
#     print("Usage: " + sys.argv[0] + " <fixedImageFile> <movingImageFile>")
#     sys.exit(1)
# fixedImageFile = sys.argv[1]
# movingImageFile = sys.argv[2]
# outputImageFile = "." + movingImageFile.split('.')[1] + "_registered." + movingImageFile.split('.')[2]
# differenceImageAfterFile = "." + movingImageFile.split('.')[1] + "_difference_after." + movingImageFile.split('.')[2]
# differenceImageBeforeFile = "." + movingImageFile.split('.')[1] + "_difference_before." + movingImageFile.split('.')[2]

base = "/content/drive/MyDrive/Registration/"
fixedImageFile = base + "VF-MRT1-1014-1174.vtk"
movingImageFile = base + "VF-MRT2-1014-1174.vtk"
outputImageFile = base + "registered.vtk"
differenceImageAfterFile = base +  "difference_after.vtk"
differenceImageBeforeFile = base + "difference_before.vtk"

In [ ]:
PixelType = itk.F

fixedImage = itk.imread(fixedImageFile, PixelType)
movingImage = itk.imread(movingImageFile, PixelType)

Dimension = fixedImage.GetImageDimension()

FixedImageType = itk.Image[PixelType, Dimension]
MovingImageType = itk.Image[PixelType, Dimension]

TransformType = itk.TranslationTransform[itk.D, Dimension]
initialTransform = TransformType.New()

print(Dimension)
print(fixedImage.shape)
print(FixedImageType)
print(MovingImageType)

3
(33, 256, 256)
<class 'itk.itkImagePython.itkImageF3'>
<class 'itk.itkImagePython.itkImageF3'>


In [ ]:
optimizer = itk.RegularStepGradientDescentOptimizerv4.New(LearningRate=4,MinimumStepLength=0.001,RelaxationFactor=0.5,NumberOfIterations=200)
metric = itk.MeanSquaresImageToImageMetricv4[FixedImageType, MovingImageType].New()


In [ ]:
registration = itk.ImageRegistrationMethodv4[FixedImageType,MovingImageType].New(FixedImage=fixedImage,
                                                                                 MovingImage=movingImage,
                                                                                 Metric=metric,
                                                                                 Optimizer=optimizer,
                                                                                 InitialTransform=initialTransform)

In [ ]:
movingInitialTransform = TransformType.New()
initialParameters = movingInitialTransform.GetParameters()
initialParameters[0] = 0
initialParameters[1] = 0
movingInitialTransform.SetParameters(initialParameters)
registration.SetMovingInitialTransform(movingInitialTransform)

identityTransform = TransformType.New()
identityTransform.SetIdentity()
registration.SetFixedInitialTransform(identityTransform)

In [ ]:

#------------------------------------
registration.SetNumberOfLevels(1)
registration.SetSmoothingSigmasPerLevel([0])
registration.SetShrinkFactorsPerLevel([1])
#------------------------------------

registration.Update()

transform = registration.GetTransform()
finalParameters = transform.GetParameters()
translationAlongX = finalParameters.GetElement(0)
translationAlongY = finalParameters.GetElement(1)

numberOfIterations = optimizer.GetCurrentIteration()

bestValue = optimizer.GetValue()

print("Result = ")
print(" Translation X = " + str(translationAlongX))
print(" Translation Y = " + str(translationAlongY))
print(" Iterations    = " + str(numberOfIterations))
print(" Metric value  = " + str(bestValue))

CompositeTransformType = itk.CompositeTransform[itk.D, Dimension]
outputCompositeTransform = CompositeTransformType.New()
outputCompositeTransform.AddTransform(movingInitialTransform)
outputCompositeTransform.AddTransform(registration.GetModifiableTransform())



Result = 
 Translation X = -0.11694292587329144
 Translation Y = -9.550855653275313
 Iterations    = 24
 Metric value  = 9197.5555730383


In [ ]:
resampler = itk.ResampleImageFilter.New(Input=movingImage,
        Transform=outputCompositeTransform,
        UseReferenceImage=True,
        ReferenceImage=fixedImage)
resampler.SetDefaultPixelValue(100)

OutputPixelType = itk.ctype('unsigned char')
OutputImageType = itk.Image[OutputPixelType, Dimension]

caster = itk.CastImageFilter[FixedImageType,
        OutputImageType].New(Input=resampler)

writer = itk.ImageFileWriter.New(Input=caster, FileName=outputImageFile)
writer.SetFileName(outputImageFile)
writer.Update()

In [ ]:
difference = itk.SubtractImageFilter.New(Input1=fixedImage,
        Input2=resampler)

intensityRescaler = itk.RescaleIntensityImageFilter[FixedImageType,
        OutputImageType].New(
            Input=difference,
            OutputMinimum=itk.NumericTraits[OutputPixelType].min(),
            OutputMaximum=itk.NumericTraits[OutputPixelType].max())

resampler.SetDefaultPixelValue(1)
writer.SetInput(intensityRescaler.GetOutput())
writer.SetFileName(differenceImageAfterFile)
writer.Update()

resampler.SetTransform(identityTransform)
writer.SetFileName(differenceImageBeforeFile)
writer.Update()


exit()